In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
# 建立权重函数
def weight(shape):
    w_stddev = 1./ tf.sqrt(shape[0] / 2.)
    return tf.random_normal(shape=shape, stddev=w_stddev)

In [3]:
###定义模型的一些参数###
X = tf.placeholder("float",shape=[None, 784])

D_W1 = tf.Variable(weight([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(weight([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
    
theta_D = [D_W1, D_W2, D_b1, D_b2]

Z = tf.placeholder("float", shape=[None, 100])

G_W1 = tf.Variable(weight([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(weight([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))##############这里我觉得是有办法可以写的更清楚的

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [4]:
#定义生成器
def generator(z):
    
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)

    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2  
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [5]:
#定义判别器
def discriminator(x):

    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)

    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    return D_prob, D_logit

In [6]:
#该函数用于生成图片（绘画）
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [7]:
#把生成器生成的样本命名为G_sample
G_sample = generator(Z)

#分别将 真实图片 和 生成图片 给判别器，输出真实的输出，虚假的输出，
#目的是为了写损失函数让两者尽量区分开来
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)


In [8]:
#####写损失函数#####
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))


In [ ]:
###损失函数的另一种写法###
# 使用交叉熵作为判别器和生成器的损失函数，因为sigmoid_cross_entropy_with_logits内部会对预测输入执行Sigmoid函数，
#所以取判别器最后一层未使用激活函数的值，即D_h1*D_W2+D_b2。
#tf.ones_like(D_logit_real)创建维度和D_logit_real相等的全是1的标注，真实图片。
#D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
#D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))

#损失函数为两部分，即E[log(D(x))]+E[log(1-D(G(z)))]，将真的判别为假和将假的判别为真
#D_loss = D_loss_real + D_loss_fake

#同样使用交叉熵构建生成器损失函数
#G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [9]:
#####写优化器######
#关键字var_list表明最小化损失函数所更新的权重矩阵！！
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

## 开始训练

In [10]:
#设置批量大小 和 生成器输入的噪声维度
batchSize = 128
Z_dim =100

#读取数据集MNIST，并放在当前目录data文件夹下MNIST文件夹中，如果该地址没有数据，则下载数据至该文件夹
mnist = input_data.read_data_sets("./MNIST/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
#看是否存在该文件夹，不存在就创建
import os

if not os.path.exists('out/'):
    os.makedirs('out/')

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    i = 0
    for it in range(20000):
        #next_batch抽取下一个批量的图片，该方法返回一个矩阵，即shape=[batchSize，784]，每一行是一张图片，共批量大小行
        batch_x, batch_y = mnist.train.next_batch(batchSize)
        
        #投入数据并根据优化方法迭代一次，计算损失后返回损失值
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: batch_x, Z: np.random.uniform(-1., 1., size=[batchSize, Z_dim])})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: np.random.uniform(-1., 1., size=[batchSize, Z_dim])})


        #每迭代2000次输出迭代数、生成器损失和判别器损失
        if it % 2000 == 0:
            print('Iter: {}'.format(it))
            print('D loss: {:.4}'. format(D_loss_curr))
            print('G_loss: {:.4}'.format(G_loss_curr))
            print()
            
        #每迭代2000次画一次图片            
        if it % 2000 == 0:
            samples = sess.run(G_sample, feed_dict={Z: np.random.uniform(-1., 1., size=[16, Z_dim])})

            fig = plot(samples)
            plt.savefig('out/me{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

Iter: 0
D loss: 1.271
G_loss: 2.391

Iter: 2000
D loss: 0.03341
G_loss: 5.654



KeyboardInterrupt: 